# Factor Component Analysis

Haytham Mohamed - INFS890 - Spring 2020

Features reduction using FCA for order data. Using factor_analyzer.factor_analyzer module

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
import seaborn as sns
from sklearn import preprocessing
from datetime import datetime
from scipy import stats
from sklearn.preprocessing import StandardScaler , MinMaxScaler
from sklearn.decomposition import PCA

sns.set()
sns.set(color_codes=True)
#sns.set_color_codes()

pd.options.display.max_rows = 15
pd.options.display.float_format = '{:,.3f}'.format

In [2]:
# 1- read processed file

home_dir = '/Users/hmohamed/github/data-research-spring2020/sock-shop'

file_dir = home_dir + '/processed-data/'
#data_file = 'order_flow_standardized_data.csv'
data_file = 'order_flow_normalized_data.csv'

SCALE_TARGETS = True
SCALE_FEATURES = True

save=True


In [3]:
def read_df(file_dir, data_file):
    df = pd.read_csv(file_dir + data_file)
    return to_time_series(df)

def to_time_series(df, index_col_name='date'):
    df[index_col_name] = pd.to_datetime(df[index_col_name])
    df.set_index(index_col_name, inplace=True)
    df.sort_index(inplace=True)
    return df

def merge(df, series):
    return pd.merge_asof(df, series, left_index=True, right_index=True, tolerance=pd.Timedelta('1 second')).bfill()    

In [4]:
normalized_data = read_df(file_dir, data_file)
normalized_data.head(5) 

,front-end_cpu_use,orders_cpu_use,orders-db_cpu_use,user_cpu_use,user-db_cpu_use,shipping_cpu_use,payment_cpu_use,carts_cpu_use,carts-db_cpu_use,front-end_pods,...,user-db_net_use,shipping_net_use,payment_net_use,carts_net_use,carts-db_net_use,nodes_cpu_use,nodes_disk_io,nodes_net_use,orders_req,orders_ltcy
date,,,,,,,,,,,,,,,,,,,,,
2020-03-26 19:48:22,0.070,0.020,0.010,0.020,0.020,0.010,0.010,0.120,0.160,1.000,...,9.158,4.263,3.363,6.915,5.557,1.180,0.000,1.785,0.400,0.400
2020-03-26 19:48:37,0.060,0.030,0.010,0.020,0.020,0.010,0.010,0.140,0.170,1.000,...,8.234,4.866,3.140,9.726,7.188,1.180,0.000,1.750,0.440,0.390
2020-03-26 20:03:22,0.210,0.060,0.020,0.050,0.030,0.020,0.010,0.240,0.660,1.000,...,25.547,3.115,4.425,29.580,7.430,1.840,0.260,3.691,1.580,0.420
2020-03-26 20:03:37,0.220,0.060,0.020,0.060,0.040,0.020,0.010,0.210,0.660,1.000,...,26.482,7.208,4.452,25.258,15.212,1.830,0.260,3.796,1.580,0.410
2020-03-26 20:03:52,0.210,0.060,0.010,0.050,0.040,0.020,0.010,0.150,0.660,1.000,...,18.687,6.874,6.016,24.157,16.161,1.750,0.000,3.837,1.490,0.390


In [5]:
normalized_data.shape

(3175, 29)

In [6]:
targets = normalized_data['orders_ltcy']
standardized_inputs = normalized_data.drop(['orders_ltcy'], axis=1)
cols = standardized_inputs.columns.values

# scale targets for better convergence
if SCALE_TARGETS:
    y_scaler =  MinMaxScaler()
    targets = y_scaler.fit_transform(targets.values.reshape(-1,1))
    
if SCALE_FEATURES:
    x_scaler = StandardScaler()
    standardized_inputs = x_scaler.fit_transform(standardized_inputs)


standardized_inputs.shape


(3175, 28)

# Multicollinearity

Use Variance Inflation Factor (VIF) from the statmodels. VIF measures how big is the square root of the standard error is compared to the case there is no multicollinearity between the variables. 
Conventionally:

$VIF = 1$ means no multicollinearity

$1< VIF < 5$ perfectly okay

$10 < VIF$ unacceptable range (some times < 6 or 8)

In [7]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

standardized_inputs = pd.DataFrame(standardized_inputs, columns=cols)
vif_inputs = standardized_inputs

vif_inputs = vif_inputs.drop('user_cpu_use', 1)
vif_inputs = vif_inputs.drop('front-end_cpu_use', 1)
vif_inputs = vif_inputs.drop('user-db_cpu_use', 1)


vif = pd.DataFrame()
vif['features'] = [col for col in vif_inputs.columns.values]
vif['VIF'] = [variance_inflation_factor(vif_inputs.values,i) for i in range(vif_inputs.shape[1])]
vif

/opt/anaconda3/envs/infs890/lib/python3.7/site-packages/statsmodels/stats/outliers_influence.py:193: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


,features,VIF
0,orders_cpu_use,14.454
1,orders-db_cpu_use,2.700
2,shipping_cpu_use,2.639
3,payment_cpu_use,6.070
4,carts_cpu_use,6.709
...,...,...
20,carts-db_net_use,11.574
21,nodes_cpu_use,57.557
22,nodes_disk_io,5.235
23,nodes_net_use,47.576


In [8]:
standardized_inputs = standardized_inputs.drop('user_cpu_use', 1)
standardized_inputs = standardized_inputs.drop('front-end_cpu_use', 1)
standardized_inputs = standardized_inputs.drop('user-db_cpu_use', 1)

vif values are OK. Consider using PCA or apply feature selection method to reduce dimentionality when using a ML model.

# Correlation Matrix

In [9]:
corr = standardized_inputs.corr()
corr.style.background_gradient(cmap='coolwarm').set_precision(3)

,orders_cpu_use,orders-db_cpu_use,shipping_cpu_use,payment_cpu_use,carts_cpu_use,carts-db_cpu_use,front-end_pods,orders_pods,user_pods,shipping_pods,payment_pods,carts_pods,front-end_net_use,orders_net_use,orders-db_net_use,user_net_use,user-db_net_use,shipping_net_use,payment_net_use,carts_net_use,carts-db_net_use,nodes_cpu_use,nodes_disk_io,nodes_net_use,orders_req
orders_cpu_use,1.000,0.396,0.341,-0.175,0.649,0.693,-0.583,-0.583,-0.583,-0.583,-0.583,-0.583,0.859,0.866,0.673,0.864,0.854,0.597,0.496,0.755,0.739,0.871,0.434,0.875,0.871
orders-db_cpu_use,0.396,1.000,0.251,0.197,-0.044,0.726,-0.171,-0.172,-0.172,-0.172,-0.172,-0.172,0.313,0.517,0.461,0.288,0.293,0.420,0.330,0.632,0.648,0.534,0.132,0.330,0.481
shipping_cpu_use,0.341,0.251,1.000,0.304,-0.002,0.304,0.144,0.143,0.143,0.143,0.143,0.143,0.009,0.190,0.160,0.022,0.004,0.253,0.193,0.273,0.272,0.248,0.129,-0.006,0.156
payment_cpu_use,-0.175,0.197,0.304,1.000,-0.229,0.061,0.757,0.758,0.758,0.758,0.758,0.758,-0.322,-0.042,-0.010,-0.273,-0.331,0.271,0.287,0.061,0.024,-0.012,-0.041,-0.345,-0.111
carts_cpu_use,0.649,-0.044,-0.002,-0.229,1.000,0.246,-0.383,-0.382,-0.382,-0.382,-0.382,-0.382,0.741,0.594,0.464,0.745,0.710,0.521,0.520,0.471,0.408,0.570,0.243,0.748,0.647
carts-db_cpu_use,0.693,0.726,0.304,0.061,0.246,1.000,-0.411,-0.412,-0.412,-0.412,-0.412,-0.412,0.635,0.766,0.621,0.578,0.578,0.571,0.430,0.901,0.896,0.873,0.307,0.638,0.780
front-end_pods,-0.583,-0.171,0.144,0.757,-0.383,-0.411,1.000,1.000,1.000,1.000,1.000,1.000,-0.670,-0.489,-0.436,-0.610,-0.668,-0.140,-0.066,-0.410,-0.445,-0.513,-0.368,-0.703,-0.524
orders_pods,-0.583,-0.172,0.143,0.758,-0.382,-0.412,1.000,1.000,1.000,1.000,1.000,1.000,-0.670,-0.489,-0.434,-0.610,-0.668,-0.137,-0.063,-0.410,-0.446,-0.513,-0.367,-0.702,-0.526
user_pods,-0.583,-0.172,0.143,0.758,-0.382,-0.412,1.000,1.000,1.000,1.000,1.000,1.000,-0.670,-0.489,-0.434,-0.610,-0.668,-0.137,-0.063,-0.410,-0.446,-0.513,-0.367,-0.702,-0.526
shipping_pods,-0.583,-0.172,0.143,0.758,-0.382,-0.412,1.000,1.000,1.000,1.000,1.000,1.000,-0.670,-0.489,-0.434,-0.610,-0.668,-0.137,-0.063,-0.410,-0.446,-0.513,-0.367,-0.702,-0.526


In [10]:
def plot_corr(df,size=10):
    '''Function plots a graphical correlation matrix for each pair of columns in the dataframe.

    Input:
        df: pandas DataFrame
        size: vertical and horizontal size of the plot'''

    corr = df.corr()
    fig, ax = plt.subplots(figsize=(size, size))
    ax.matshow(corr)
    plt.xticks(range(len(corr.columns)), corr.columns);
    plt.yticks(range(len(corr.columns)), corr.columns);
    
#plot_corr(standardized_inputs,20)    

The function most_highly_correlated() will print out the linear correlation coefficients for each pair of variables in your data set, in order of the correlation coefficient. This lets you see very easily which pair of variables are most highly correlated.

In [11]:
def most_highly_correlated(mydataframe, numtoreport):
    # find the correlations
    cormatrix = mydataframe.corr()
    # set the correlations on the diagonal or lower triangle to zero,
    # so they will not be reported as the highest ones:
    cormatrix *= np.tri(*cormatrix.values.shape, k=-1).T
    # find the top n correlations
    cormatrix = cormatrix.stack()
    cormatrix = cormatrix.reindex(cormatrix.abs().sort_values(ascending=False).index).reset_index()
    # assign human-friendly names
    cormatrix.columns = ["FirstVariable", "SecondVariable", "Correlation"]
    return cormatrix.head(numtoreport)

mcdf = most_highly_correlated(standardized_inputs, 30)
mcdf = mcdf[mcdf.Correlation > 0.5]

mcdf

,FirstVariable,SecondVariable,Correlation
0,orders_pods,carts_pods,1.000
1,orders_pods,user_pods,1.000
2,user_pods,carts_pods,1.000
3,shipping_pods,carts_pods,1.000
4,orders_pods,shipping_pods,1.000
...,...,...,...
25,orders_net_use,orders_req,0.890
26,orders_net_use,user_net_use,0.885
27,carts_net_use,nodes_cpu_use,0.879
28,orders_net_use,nodes_net_use,0.877


# Component Factor Analysis

### Adequacy Test

Check adquacy of the correlation matrix for factor analysis. Checking both Bartlett's test if the correlation matrix is collectively significant to include factors, and checking the Measure of Adquacy Samplin (MSA), a.k.a Kaiser-Meyer-Olkin (KMO) to check the pattern

In [12]:
from factor_analyzer.factor_analyzer import calculate_bartlett_sphericity

# Bartlett ’s test
chi_square_value,p_value=calculate_bartlett_sphericity(standardized_inputs)

chi_square_value, p_value

/opt/anaconda3/envs/infs890/lib/python3.7/site-packages/factor_analyzer/factor_analyzer.py:118: RuntimeWarning: invalid value encountered in log
  statistic = -np.log(corr_det) * (n - 1 - (2 * p + 5) / 6)


(nan, nan)

In this Bartlett’s test, the p-value is 0. The test was statistically significant, indicating that the observed correlation matrix is not an identity matrix.

In [13]:
from factor_analyzer.factor_analyzer import calculate_kmo

kmo_all,kmo_model=calculate_kmo(standardized_inputs)

kmo_model

/opt/anaconda3/envs/infs890/lib/python3.7/site-packages/factor_analyzer/utils.py:248: UserWarning: The inverse of the variance-covariance matrix was calculated using the Moore-Penrose generalized matrix inversion, due to its determinant being at or very close to zero.
  warnings.warn('The inverse of the variance-covariance matrix '


0.8304083248850301

The overall KMO for our data is 0.785, which is excellent ( > 0.5). This value indicates that you can proceed with your planned factor analysis.

In [14]:
kmo_all

array([0.91932656, 0.95656819, 0.53773348, 0.9421507 , 0.85797966,
       0.8269288 , 0.65410993, 0.65289131, 0.65289131, 0.65289131,
       0.65289131, 0.65289131, 0.95958876, 0.97579045, 0.97933118,
       0.97344029, 0.98271998, 0.92539533, 0.90915303, 0.94675026,
       0.95754935, 0.86240024, 0.66755695, 0.94441117, 0.95081742])

Features KMO values are significant ( > 0.5). It would be ok to consider them all to analyze the factors.

### Number of Factors

For choosing the number of factors, you can use the Kaiser criterion and scree plot. Both are based on eigenvalues.

In [15]:
# Create factor analysis object and perform factor analysis
from factor_analyzer.factor_analyzer import FactorAnalyzer

fa = FactorAnalyzer(n_factors=25, rotation=None)
fa.fit(standardized_inputs)
# Check Eigenvalues
ev, v = fa.get_eigenvalues()
ev

LinAlgError: Singular matrix

Here, you can see only for 2 factors eigenvalues are greater than one. It means we need to choose only 2 factors (or unobserved variables). Also we can draw a scree plot to check how many components to consider. Number of factors will be for the one spotted with Eigenvalue >= 1

In [ ]:
# Create scree plot using matplotlib
plt.scatter(range(1,standardized_inputs.shape[1]+1),ev)
plt.plot(range(1,standardized_inputs.shape[1]+1),ev)
plt.title('Scree Plot')
plt.xlabel('Factors')
plt.ylabel('Eigenvalue')
plt.grid()
plt.show()

### Component Factor Loadings

Choosing 2 factors however segregate the features into three interpreted (logical) factors. Looking the loading below we see the features loaded as follows:

Factor 1:  svc_cpu_use, svc_cpu_thr, svc_net_use, svc_disk_use, system_net_use, svc_req_size, svc_resp_size, svc_req_rate ==> a factor about svc_usage_tolerance

Factor 2: system_cpu_use and system_cpu_sat, svc_pods ==> a factor about system_workload_tolerance

The new components are just the two main dimensions of variation


In [ ]:
number_of_factors = 4
fa = FactorAnalyzer(n_factors=number_of_factors, rotation="promax")  # promax rotation eliminate cross loadings
fa.fit(standardized_inputs)

factors_heading = []
for i in range(number_of_factors):
    factors_heading.append('F' + str(i+1))

variables = standardized_inputs.columns.values
loadings = pd.DataFrame(fa.loadings_, index=variables, columns=factors_heading)

loadings

Explained Variance

In [ ]:
indx = ['SS Loadings', 'Proportion Variance', 'Cumulative Variance']
explained_var = pd.DataFrame(fa.get_factor_variance(), columns=factors_heading, index=indx)
explained_var

Total 55.3% cumulative Variance explained by the 2 factors

### Factored Data

To calculate the values of a principal component, we can define our own function to calculate a principal component given the loadings and the input variables’ values:

In [ ]:
def calcpc(variables, loadings):
    # find the number of samples in the data set and the number of variables
    numsamples, numvariables = variables.shape
    # make a vector to store the component
    pc = np.zeros(numsamples)
    # calculate the value of the component for each sample
    for i in range(numsamples):
        valuei = 0
        for j in range(numvariables):
            valueij = variables.iloc[i, j]
            loadingj = loadings[j]
            valuei = valuei + (valueij * loadingj)
        pc[i] = valuei
    return pc

In [ ]:
# Factor data of the 3 factors

loadings_df = fa.loadings_.reshape(-1,number_of_factors)

pc1_data = calcpc(pd.DataFrame(standardized_inputs), loadings_df[:,0])
pc1_data = pd.DataFrame(pc1_data, columns=['svc_shopping_cpu_use']).round(3)

pc2_data = calcpc(pd.DataFrame(standardized_inputs), loadings_df[:,1])
pc2_data = pd.DataFrame(pc2_data,  columns=['system_inferance_cpu_use']).round(3)

pc3_data = calcpc(pd.DataFrame(standardized_inputs), loadings_df[:,2])
pc3_data = pd.DataFrame(pc3_data,  columns=['svc_shipping_cpu_use']).round(3)

pc4_data = calcpc(pd.DataFrame(standardized_inputs), loadings_df[:,3])
pc4_data = pd.DataFrame(pc4_data,  columns=['svc_payment_cpu_use']).round(3)

targets = pd.DataFrame(targets.values, columns=['orders_ltcy'])

factor_data = pd.concat([pc1_data, pc2_data, pc3_data, pc4_data, targets.round(3)], axis=1, sort=False)
factor_data.shape


In [ ]:
factor_data.head()

In [ ]:
# Save Data to a file
if save:
    factor_data.to_csv(path_or_buf=file_dir + 'order_flow_factored_data.csv', index=False)